![](Figure_1.png)

## Derivative of tanh

$$\tanh x = { \frac{\sinh x}{\cosh x} }
 = { \frac {e^{x}-e^{-x}}{e^{x} +e^{-x}} } $$


$$\frac{d}{dx}{\sinh x} = \frac{1}{2}*(\frac{d}{dx}e^{x}-\frac{d}{dx}e^{-x}) = \frac{1}{2}*(e^{x} + e^{-x}) = \cosh x $$
$$\frac{d}{dx}{\cosh x} = \frac{1}{2}*(\frac{d}{dx}e^{x}+\frac{d}{dx}e^{-x}) = \frac{1}{2}*(e^{x} - e^{-x}) = \sinh x. $$
<p style="text-align: center;">  
Applying the quotient rule:
</p>

$$ \frac{d}{dx}{\tanh x} = \frac{\cosh x \frac{d}{dx} \sinh x - \sinh x \frac{d}{dx} \cosh x } {{\cosh}^{2} x} $$

$$ = \frac{\cosh x \cosh x - \sinh x \sinh x}{\cosh^{2} x}$$

$$ = 1 - \frac{\sinh ^{2}x}{\cosh^{2}x} = 1 - \tanh^{2} x $$

$$ \blacksquare . $$




## Derivative of logistic sigmoid

$$ \sigma (x) =  \frac{1}{1 + e^{-x}}$$

<p style="text-align: center;">  
Applying the chain rule:
</p>
$$ \frac{d}{dx} \sigma (x) = - \frac{1}{(1 + e^{-x})^{2}} * -e^{-x} $$

$$ = \sigma (x) \frac{e^{-x}}{1 + e^{-x}} $$

$$ = \sigma (x) (1 - \sigma (x)) $$

$$ \blacksquare . $$



## Derivative of ReLU

The Rectified Linear Unit is defined as $ f(x)=\text{max}(0,x) $.  
We can rewrite this using two cases:

\begin{equation} 
f(x)=
    \begin{cases}
      x, & \text{if}\ x>0 \\
      0, & \text{otherwise}
    \end{cases}
\end{equation} .

Upon simple differentiation of the two cases, we get

\begin{equation} 
f'(x)=
    \begin{cases}
      1, & \text{if}\ x>0 \\
      0, & \text{otherwise}
    \end{cases}
\end{equation} 

The ReLU is discontinuous at x = 0, 
therefore its derivative at x = 0 is technically not defined;
however, we are explicitly setting $ f'(0) = 0 $ in the statement above, so we have defined a derivative $\forall x \in \mathbb{R}$ 



In [1]:
x = 'asdf'